In [23]:
from sklearn.neural_network import MLPRegressor

MLP = MLPRegressor(hidden_layer_sizes=(50, ), activation='relu', algorithm='adam', alpha=0.0001, 
                   batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
                   max_iter=100, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False,
                   momentum=0.9, nesterovs_momentum=True, early_stopping=False,validation_fraction=0.1,
                   beta_1=0.9, beta_2=0.999, epsilon=1e-08)


In [24]:
X_TRAIN = np.load('../output/bar_X_TRAIN.npy')
Y_TRAIN = np.load('../output/bar_Y_TRAIN.npy')

X_TEST = np.load('../output/bar_X_TEST.npy')
Y_TEST = np.load('../output/bar_Y_TEST.npy')

n_samples = 10000

def getRMS_error(model, X, Y): 
    Y_predict = model.predict(X)
    MSE = (Y-Y_predict)**2
    RMS_errors = np.sqrt(np.average(MSE))
    return RMS_errors, MSE 

MLP = MLP.fit(X_TRAIN[:n_samples], Y_TRAIN[:n_samples])


RMS_train, MSE_train = getRMS_error(MLP, X_TRAIN[:n_samples], Y_TRAIN[:n_samples])
RMS_test, MSE_test = getRMS_error(MLP, X_TEST[:n_samples], Y_TEST[:n_samples])
print 'RMS Training Error', RMS_train
print 'RMS Test Error', RMS_test
#neural_network.

RMS Training Error 2.55777373011
RMS Test Error 419.82790616


In [ ]:
# Run `pip install sigopt` to download the python API client
import math
import sigopt
conn = sigopt.Connection(client_token="BPNMNMBUFKVDKJDUEBULARYOGQQAUQFFYEEQQPQHCQZQUNAJ")

# Create an experiment with two parameters, x and y
experiment = conn.experiments().create(
  name="RF Optimization",
  parameters=[
    {'name': 'x', 'bounds': {'max': 1, 'min': 0}, 'type': 'double', 'precision' : 4},
    {'name': 'y', 'bounds': {'max': 1, 'min': 0}, 'type': 'double', 'precision' : 4},
  ],
)
print("Created an experiment with id {0}.".format(experiment.id))

# Receive a suggestion from SigOpt and evaluate our example function
# Franke function - http://www.sfu.ca/~ssurjano/franke2d.html
def evaluate_metric(x, y):
  e1 = - (math.pow((9 * x - 2), 2)/4) - (math.pow((9 * y - 2), 2)/4)
  e2 = - (math.pow((9 * x + 1), 2)/49) - ((9 * y + 1)/10 )
  e3 = - (math.pow((9 * x - 7), 2)/4) - (math.pow((9 * y - 3), 2)/4)
  e4 = - (math.pow((9 * x - 4), 2)) - (math.pow((9 * y - 7), 2))

  return (0.75 * math.exp(e1) + 0.75 * math.exp(e2) + 0.5 * math.exp(e3) - 0.2 * math.exp(e4))






# In a loop: receive a suggestion, evaluate our metric, then report an observation
for _ in range(20):
  suggestion = conn.experiments(experiment.id).suggestions().create()
  x = suggestion.assignments['x']
  y = suggestion.assignments['y']
  print("Received a suggestion with x={x} and y={y}.".format(x=x, y=y))
  value = evaluate_metric(x, y)
  print("The function evaluated to {0} using these parameters.".format(value))
  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion.id,
    value=value,
  )